### Import

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from openai import OpenAI
import requests
import torch
import re
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Practicum/yake_results_t2_preprocessed.csv", index_col=0)

In [ ]:
df.head()

,Name,Final_description,Keywords,Final_keywords
0,HAICViz - MuGSI,haicviz mugsi the healthcareassociated infect...,"[('targeting hai prevention', 0.00202504458285...","['targeting hai prevention', 'cdcs eip engages..."
1,HHS Data Governance Board Membership - w3i2-ja...,hhs data governance board membership wijayx ...,"[('governance board membership', 0.00121911179...","['governance board membership', 'data governan..."
2,Drug Interaction API - 5pe7-4rei - Archive Rep...,drug interaction api perei archive repositor...,"[('drug interaction api', 0.001365041929041524...","['drug interaction api', 'interaction api pere..."
3,NCHS Research and Development Survey (RANDS) R...,nchs research and development survey rands rou...,"[('subnationally representative estimates', 0....","['subnationally representative estimates', 'cd..."
4,"NNDSS - TABLE 1AA. Poliovirus infection, nonp...",nndss table aa poliovirus infection nonparal...,"[('poliovirus infection nonparalytic', 0.00078...","['poliovirus infection nonparalytic', 'psittac..."


In [ ]:
df["Keywords"].values[:1]

array(["[('targeting hai prevention', 0.0020250445828521087), ('cdcs eip engages', 0.002307788961357151), ('academic medical center', 0.002307788961357151), ('medical center partners', 0.002307788961357151), ('answer critical questions', 0.002307788961357151), ('shaping future policies', 0.002307788961357151), ('healthcareassociated infection component', 0.0030509113400522467), ('state health departments', 0.0030509113400522467), ('emerging hai threats', 0.0030509113400522467), ('hai threats advanced', 0.0030509113400522467), ('advanced infection tracking', 0.0030509113400522467), ('infection tracking methods', 0.0030509113400522467), ('united states information', 0.0030509113400522467), ('states information gathered', 0.0030509113400522467), ('threats advanced infection', 0.0030509113400522475), ('recommendations targeting hai', 0.0030509113400522475), ('haicviz mugsi', 0.011491011006771688), ('hai prevention', 0.015099791981184588), ('mugsi the healthcareassociated', 0.01721071406988

In [ ]:
df["Final_keywords"].values[:1]

array(["['targeting hai prevention', 'cdcs eip engages', 'academic medical center', 'medical center partners', 'answer critical questions', 'shaping future policies', 'healthcareassociated infection component', 'state health departments', 'emerging hai threats', 'hai threats advanced', 'advanced infection tracking', 'infection tracking methods', 'united states information', 'states information gathered', 'threats advanced infection', 'recommendations targeting hai', 'haicviz mugsi', 'hai prevention', 'mugsi the healthcareassociated', 'component of cdcs']"],
      dtype=object)

In [ ]:
df["Final_keywords"].values[:1][::-1]

array(["['targeting hai prevention', 'cdcs eip engages', 'academic medical center', 'medical center partners', 'answer critical questions', 'shaping future policies', 'healthcareassociated infection component', 'state health departments', 'emerging hai threats', 'hai threats advanced', 'advanced infection tracking', 'infection tracking methods', 'united states information', 'states information gathered', 'threats advanced infection', 'recommendations targeting hai', 'haicviz mugsi', 'hai prevention', 'mugsi the healthcareassociated', 'component of cdcs']"],
      dtype=object)

In [ ]:
df['Final_keywords_reversed'] = df['Final_keywords'].apply(lambda x: eval(x)[::-1])

In [ ]:
# Sort the tags in decreasing order of probabilities
df["Final_keywords_reversed"].head()

0    [component of cdcs, mugsi the healthcareassoci...
1    [data, dataset hhs, dataset tracks, archive re...
2    [api, drug, repository for previous, interacti...
3    [representative estimates, nchs research, deve...
4    [poliovirus, table, dataset nndss, dataset tra...
Name: Final_keywords_reversed, dtype: object

### GPT4-Turbo

In [ ]:
client = OpenAI(api_key="sk-proj-OW3BIPRXsb4uhiwB1vO9T3BlbkFJNBekzWtxf16Wl2qBHmVS")
print("Connection established")


response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are an expert in creating effective prompts for generating tags based on Name and Final_description columns, and you need to improve current tags"},
        {"role": "user", "content": "The dataset contains three columns: Name, Final_description, and Final_keywords. The Final_keywords column contains tags generated using a statistical model, listed in reverse order of importance, meaning the last tag is the most relevant. However, our client is not satisfied with the current tags and has requested improvements. The client has specified that the improved tags should be able to identify and create tags for terms used in medical journals. I will be generating tags based on descriptions. Please recommend 5 effective prompts to generate the best tags from a given description. And the prompt need to have this sentence : ' This is example of tags based on name and description: {tags}. Improve these tags based on the following descriptions and select the top 5 tags: {desc}'"}
    ]
)

recommended_prompts = response.choices[0].message.content
print("Recommended Prompts:\n", recommended_prompts)


Connection established
Recommended Prompts:
 To effectively generate improved tags that meet the client's needs, particularly for terms used in medical journals, your prompts must be structured to guide a language model in focusing on the medical and scientific relevance of the terms within the descriptions. Here are five effective prompts you can use to optimize the generation of tags:

1. **Prompt Emphasizing Specific Medical Terms**  
   "This is an example of tags based on name and description: {tags}. Improve these tags based on the following descriptions, focusing specifically on medically relevant terms and their applications. Identify and select the top 5 tags that are most crucial for a medical journal context: {desc}"

2. **Prompt for Highlighting Research and Therapy Related Terms**  
   "This is an example of tags based on name and description: {tags}. Enhance these tags by extracting key elements from the description related to research methods, therapeutic areas, and clin

In [ ]:
prompts = [
    "This is an example of tags based on name and description: {tags}. Improve these tags based on the following descriptions, ensuring technical accuracy and relevance to medical terminology. Emphasize terms that align with recent medical research trends. Select the top 5 tags. Format the response as a comma-separated list of tags: {desc}", "This is an example of tags based on name and description: {tags}. Using the following descriptions, refine these tags to enhance specificity. Focus on naming specific diseases, treatments, or medical conditions mentioned. Select the top 5 tags. Format the response as a comma-separated list of tags: {desc}", "This is an example of tags based on name and description: {tags}. Reassess these tags based on the following descriptions. Ensure that the terminology used is consistent with that used in medical journal standards. Select the top 5 tags. Format the response as a comma-separated list of tags: {desc}", "This is an example of tags based on name and description: {tags}. Improve upon these tags using the following descriptions by comparing with related terms in existing medical literature. Identify connections or advancements and select the top 5 tags. Format the response as a comma-separated list of tags: {desc}", "This is an example of tags based on name and description: {tags}. From the following descriptions, develop these tags further by identifying broad medical themes or categories they fall under. Prioritize overarching medical themes over specific terms. Select the top 5 tags. Format the response as a comma-separated list of tags: {desc}"
    ]

### Test

In [ ]:
df_a = df.head(3)
df_a

,Name,Final_description,Keywords,Final_keywords
0,HAICViz - MuGSI,haicviz mugsi the healthcareassociated infect...,"[('targeting hai prevention', 0.00202504458285...","['targeting hai prevention', 'cdcs eip engages..."
1,HHS Data Governance Board Membership - w3i2-ja...,hhs data governance board membership wijayx ...,"[('governance board membership', 0.00121911179...","['governance board membership', 'data governan..."
2,Drug Interaction API - 5pe7-4rei - Archive Rep...,drug interaction api perei archive repositor...,"[('drug interaction api', 0.001365041929041524...","['drug interaction api', 'interaction api pere..."


In [ ]:

for j in range(1, 6):
    df_a[f'Improved_tag_Prompt_{j}'] = ''


for i, (tags, desc) in enumerate(zip(df_a["Final_keywords"], df_a["Final_description"])):
    for j, prompt in enumerate(prompts, start=1):
        formatted_prompt = prompt.format(tags=tags, desc=desc)
        completion =  client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a tag generating assistant, skilled in extracting key information based on description. You are also skilled in improving generated tags"},
                {"role": "user", "content": formatted_prompt}
            ]
        )

        improved_tags = completion.choices[0].message.content.strip().replace('\n', ', ')
        tags_list = improved_tags.split(', ')[:3]  # 상위 3개의 태그만 선택
        df_a.at[i, f'Improved_tag_Prompt_{j}'] = ', '.join(tags_list)

        print(f'Prompt {j} for index {i}:')
        print(improved_tags)
        print('-------------------------------------------------')



<ipython-input-45-5e6bf7df4ad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a[f'Improved_tag_Prompt_{j}'] = ''
<ipython-input-45-5e6bf7df4ad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a[f'Improved_tag_Prompt_{j}'] = ''
<ipython-input-45-5e6bf7df4ad9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

Prompt 1 for index 0:
'healthcare-associated infection prevention', 'antibiotic resistance tracking', 'CDC's Emerging Infections Program', 'medical centers collaboration', 'infection surveillance and policy development'
-------------------------------------------------
Prompt 2 for index 0:
'hai prevention strategies', 'antibiotic resistance tracking', 'cdcs emerging infection program', 'healthcare-associated infection research', 'state health department collaboration'
-------------------------------------------------
Prompt 3 for index 0:
HAI prevention strategies, CDC's Emerging Infections Program, healthcare-associated infection surveillance, antibiotic resistance tracking, public health policy development
-------------------------------------------------
Prompt 4 for index 0:
'HAI prevention strategies', 'CDC's Emerging Infections Program', 'advanced infection tracking technology', 'healthcare-associated infection research', 'antibiotic resistance monitoring'
----------------------

In [ ]:

df_a.to_csv("improved_tags_test.csv", index=False)


display(df_a)

,Name,Final_description,Keywords,Final_keywords,Improved_tag_Prompt_1,Improved_tag_Prompt_2,Improved_tag_Prompt_3,Improved_tag_Prompt_4,Improved_tag_Prompt_5
0,HAICViz - MuGSI,haicviz mugsi the healthcareassociated infect...,"[('targeting hai prevention', 0.00202504458285...","['targeting hai prevention', 'cdcs eip engages...","'healthcare-associated infection prevention', ...","'hai prevention strategies', 'antibiotic resis...","HAI prevention strategies, CDC's Emerging Infe...","'HAI prevention strategies', 'CDC's Emerging I...","infection prevention, public health policy, an..."
1,HHS Data Governance Board Membership - w3i2-ja...,hhs data governance board membership wijayx ...,"[('governance board membership', 0.00121911179...","['governance board membership', 'data governan...","hhs_data_governance_board, wijayx_archive_mana...","hhs data governance board, wijayx archive repo...","HHS Data Governance Board Membership, Wijayx A...","hhs data governance board, membership wijayx a...","health data management, data repository govern..."
2,Drug Interaction API - 5pe7-4rei - Archive Rep...,drug interaction api perei archive repositor...,"[('drug interaction api', 0.001365041929041524...","['drug interaction api', 'interaction api pere...","drug interaction dataset, API development, met...","drug interaction dataset, archive repository, ...","drug interaction API, Perei archive, dataset t...","drug interaction database, API for drug intera...","pharmacology, pharmaceutical technology, medic..."


In [ ]:
df_a.to_csv("/content/drive/MyDrive/Practicum/GPT_Result_test.csv")


### Real

In [ ]:

for j in range(1, 6):
    df[f'Improved_tag_Prompt_{j}'] = ''


for i, (tags, desc) in enumerate(zip(df["Final_keywords"], df["Final_description"])):
    for j, prompt in enumerate(prompts, start=1):
        formatted_prompt = prompt.format(tags=tags, desc=desc)
        completion =  client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "You are a tag generating assistant, skilled in extracting key information based on description. You are also skilled in improving generated tags"},
                {"role": "user", "content": formatted_prompt}
            ]
        )

        improved_tags = completion.choices[0].message.content.strip().replace('\n', ', ')
        tags_list = improved_tags.split(', ')[:5]
        df.at[i, f'Improved_tag_Prompt_{j}'] = ', '.join(tags_list)

        print(f'Prompt {j} for index {i}:')
        print(improved_tags)
        print('-------------------------------------------------')



Prompt 1 for index 0:
'healthcare-associated infection prevention', 'CDC Emerging Infections Program', 'academic medical center collaboration', 'advanced infection tracking technologies', 'antibiotic resistance research'
-------------------------------------------------
Prompt 2 for index 0:
'healthcare-associated infection prevention', 'CDC's Emerging Infections Program', 'state health departments collaboration', 'advanced infection tracking methods', 'antibiotic resistance tracking'
-------------------------------------------------
Prompt 3 for index 0:
Here are the reassessed and refined top 5 tags based on the description provided, reformatted to align with medical journal standards:, , 'HAI prevention strategies', 'CDC EIP activities', 'emerging HAI surveillance', 'antibiotic resistance monitoring', 'health policy development'
-------------------------------------------------
Prompt 4 for index 0:
'healthcare-associated infection prevention', 'CDC's Emerging Infections Program', '

KeyboardInterrupt: 

In [ ]:

df.to_csv("improved_tags.csv", index=False)
df.to_csv("/content/drive/MyDrive/Practicum/GPT_Result.csv")



print(df)